# Workflow of MUTIS

Here we illustrate the workflow of MUTIS (class version) to analyze correlation between two signals.
The general workflow is:

- **We load the data** in a suitable way.

- **We generate the synthetic light curves**, which are used to calculate the confidence levels of correlations.
    
    This is an automatic process if the method used for generating them is just **sampling** (lc_gen_samp), or flux randomization with the same **psd** (lc_gen_psd). If we simulate them as an **stochastic** process (lc_gen_ou) as suggested for example in Tavecchio, Bonnoli and Galanti, 2020 (doi:10.1093/mnras/staa2055) we need to carefully choose the parameters for this stochastic process (currently trying to optimize it).
    
- **We generate the correlations** (the synthetic ones and the real one) and plot them together.
    
    For this we need to manually choose the time binning.

**Juan Escudero**

In [ ]:
# I'm doing a lot of changes to MUTIS while writting this, better reload automatically.
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy as sp

import pandas as pd
pd.set_option("display.max_columns", None)

from astropy.time import Time

import matplotlib as mplt
import matplotlib.pyplot as plt

import mutis

## Load data
Load it with pandas becase I like the way pandas prints tables. Also useful accesing vars as strings!

In [ ]:
data = {}

In [ ]:
data['3mm'] = pd.read_csv('data/mm-I.dat', comment='!')
data['3mm']

In [ ]:
data['gamma'] = pd.read_csv('data/gamma-I.dat', comment='!')
data['gamma']

## Show data

In [ ]:
fig, ax = plt.subplots(nrows=2, figsize=(12,4), sharex=True, gridspec_kw={'hspace':0.00})


ax[0].errorbar(data['gamma']['jyear'], data['gamma']['CFlux'], data['gamma']['CFluxErr'], fmt='b.--', label='Gamma-ray flux (0.1 to 200 GeV)', lw=1) # freq?
ax[0].set_ylabel('$\gamma/\mathrm{cm}^2/\mathrm{s}?$')
ax[0].legend()
ax[0].grid()

ax[1].errorbar(data['3mm']['jyear'], data['3mm']['I'], data['3mm']['dI'], fmt='k.--', label='mm', lw=1)  #data['3mm'].plot(x='year', y='I')
ax[1].set_ylabel('Jy')
ax[1].legend()
ax[1].grid()

fig.suptitle('Test Source Flux Density')
plt.xlabel('years')
plt.show()

## Correlate data

### Choose data and methods

Choose the signals to correlate

In [ ]:
t1, s1, ds1 = data['3mm']['jyear'], data['3mm']['I'], data['3mm']['dI']
t2, s2, ds2 = data['gamma']['jyear'], 1e6*data['gamma']['CFlux'], 1e6*data['gamma']['CFluxErr'] ### ¡Use sensible units!

Create signals with the corresponding data and choose the method for generating the light curves.

In [ ]:
sig1 = mutis.Signal(t1, s1, ds1, 'lc_gen_psd_nft')
sig2 = mutis.Signal(t2, s2, ds2, 'lc_gen_ou')

Create the correlation with the corresponding signals and choose the method for generating the correltions.

In [ ]:
correlation = mutis.Correlation(sig1, sig2, 'welsh')

### Generate synthetic light curves

 **HUMAN INTERVENTION**
 
Chose good parameters for time binning!

Good parameters will have a high and similar $n_i$ for all bins, with bins covering most of the time range.

In [ ]:
correlation.gen_times(dtmin=0.25)

In [ ]:
correlation.plot_times()

**HUMAN INTERVENTION**

If the method for generating the synthetic light curves is **lc_gen_ou** we need to chose a good set of parameters. 

In this case we have set only sig2's method to be lc_gen_ou because lc_gen_psd is more or less okey for sig1.

The Signal class includes methods to deal with this OU parameter extraction. This could be more or less an authomathic process, but for some (not a negible number of) cases degenerancy of these OU parameter makes human intervention necessary. We need:
1. To plot the histogram and check if the supposed probability density function (pdf) fits nicely (several ways: curve fitting or MLE).
2. To check if the stimates are reasonable and produce nice light curves.
3. Actually generate the light curves.

#### Plot the histogram and get extracted parameters

In [ ]:
%%time
fits = sig2.OU_fit()

In [ ]:
fits

##### Check if this parameters generate nice light curves.

In [ ]:
#%matplotlib inline
sig2.check_gen(fgen='lc_gen_ou', fgen_params=dict(theta=17.8, mu=0.14, sigma=5.9))

**Set the parameters**

In [ ]:
sig2.OU_theta, sig2.OU_mu, sig2.OU_sigma = 17.8, 0.13, 5.9

##### Actually generate the synthetic light curves

In [ ]:
%%time
correlation.gen_synth(400)

### Generate and plot correlations

In [ ]:
%%time
correlation.gen_corr();

In [ ]:
correlation.plot_corr()

### Find the peaks

In [ ]:
pd.DataFrame(correlation.peak_find())